In [39]:
import os
import sys
import urllib.request
import json
import pandas as pd
import re
import time
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup
import requests

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [2]:
# 검색어 입력
query = input("검색어 : ")


# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(2)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())

print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
page_num = 1
end_page=total_num//7+1
end_page
contents_num = 7

blog_title_lst = []
blog_url_lst = []

while contents_num == 7 : 
    search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
    driver.get(search_url)
    time.sleep(0.5)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
    
    # 블로그 제목, url 가져오기
    posts_data = soup.select('div.info_post > div.desc > a.desc_inner')

    for post in posts_data :
        title=post.get_text().replace('\n','').strip()
        href=post.attrs['href']
        blog_title_lst.append(title)
        blog_url_lst.append(href)
    
    page_num +=1
    contents_num = len(posts_data)

print(blog_title_lst)
print(blog_url_lst)

2024-04-29 2024-05-06
총 블로그글 개수 :  11,233건
연관검색어: 
강남역 피부과강남역 카페강남역 놀거리강남역 미용실강남역 혼밥강남역 브런치강남역 나세르강남역 파스타강남역 다이소강남역 고기집강남역 이자카야강남역 인질극강남역 9번출구강남역인질강남역 흉기강남역 칼부림강남역 먹거리강남역 버스강남역 행사강남역 실시간


['강남역임플란트 잘하는곳 치료 비용 비교한 후', '강남역임플란트 상실한 치아를 대신해주는 과정', '강남역임플란트 치아상실의 대처 방안으로', '강남역 피부과 리쥬란힐러 생생한 전후비교 후기', '강남역임플란트 치아를 잃어 대처하기 위해', '강남역 헬스장 몸매 가꾸고 건강 챙기기', '강남역pt 건강하게 운동해요', '강남역 네일 추천, 집중 케어 네일샵 디자인뷰티플러스', '강남역피부과추천 악센토 받고 주근깨 박멸한 후기', '강남역 피부관리, 리프팅레이저 후 수분관리 코어킴스킨', '강남역 샤브샤브 맛집 미미옥 신논현역 한식 맛집', '강남역헬스장 HIIT 프로그램으로 운동해보기!', '강남역두피관리 닥터모락에서 말끔히 관리받았어요', '[딘타이펑] 강남역 점심 맛집 딤섬', '강남역 삼겹살 가성비 코스요리처럼 구워주는 그릴링서비스 유니네고깃간', '강남역 치과 추천 사례별 치료과정', '강남역 사무실 임대 케이스퀘어 강남2', '강남역 한우 맛집 일편등심 강남점', '강남역 사무실 임대 광일프라자', '강남역 타로상담 해주는 칵테일바 타로맨스', '강남역 치과 추천 임플란트 식립과정', "강남역 삼겹살 가성비 맛집 '유니네 고깃간'", '강남역 튠페이스 합리적 가격의 청담 피부과 처럼', '[도키메이테.강남역.디너.#3] 강남역 최근접 오마카세 스시야', '강남역한식맛집 봉된장 본점 우렁된장 점심 리뷰', '강남역초역세권/래미안서초에스티지에스 84B타입(34평) 매매', '강남역치킨 맛집 도른계, 줄서서 먹는 특별한 통닭구이', '[강남역 맛집] 봉구가래떡볶이역삼점 떡볶이맛집 점심 배달 포장 가래떡볶이 메뉴추천...'

In [29]:
# 확인
print(len(blog_title_lst),len(blog_url_lst))

4000 4000


In [30]:
df = pd.DataFrame({"title":blog_title_lst, "url":blog_url_lst})
df

,title,url
0,강남역임플란트 잘하는곳 치료 비용 비교한 후,https://blog.naver.com/mkctcast/223437247000
1,강남역임플란트 상실한 치아를 대신해주는 과정,https://blog.naver.com/pjh6677/223435045671
2,강남역임플란트 치아상실의 대처 방안으로,https://blog.naver.com/happ0217/223435207647
3,강남역 피부과 리쥬란힐러 생생한 전후비교 후기,https://blog.naver.com/ycs7117/223435139372
4,강남역임플란트 치아를 잃어 대처하기 위해,https://blog.naver.com/vineyardk/223433935912
...,...,...
3995,애쉬바이올렛 염색 탈색후 염색 추천 베스트 원!,https://blog.naver.com/lovesotomusi/223436429177
3996,[신논현역 맛집] 위트앤미트 강남점 / 파스트라미 샌드우치가 맛있었던 신논현역 맛집...,https://blog.naver.com/aofl514/223436429431
3997,백소정 메뉴 마제소바 맛집 돈까스 세트 강남 내돈내산,https://blog.naver.com/arargrg/223434605386
3998,[전시] 타나카 타츠야 : 미니어처 라이프 후기,https://blog.naver.com/andrichrich/223435229982


- blog 본문 가져오기(test)

In [69]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# 해당 blog 불러오기 
url = blog_url_lst[10]
driver.get(url)
time.sleep(1)


# iframe 접근 
driver.switch_to.frame('mainFrame')


# 제목 가져오기
overlays_title = ".se-module.se-module-text.se-title-text"
html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
title = html_title.text
print(title)


# 포스팅 날짜 가져오기
overlays_publishDate = ".se_publishDate.pcol2"
html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
date = html_publishDate.text
print(date)

# 본문 가져오기 
overlays_content = ".se-component.se-text.se-l-default"
html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
contents_lst = []
for content in html_content : 
    contents_lst.append(content.text.replace('\n',''))
    

print(contents_lst)

# 해시태그 가져오기 

# title = html_title.text 
# title





['요즘 친구랑 평일 점심 맛집 탐방을다니고 있는데, 이번에 강남역과신논현역 근처에 있는 미미옥을 다녀왔다모던한 분위기의 매장과 한식 분위기 물씬한메뉴의 조화가 매력적인 곳이었다', '서울 강남구 봉은사로4길 24 1층0507~1318~1975운영시간매일 11:00 - 21:30(15:00 - 17:00 브레이크타임)21:00 라스트오더', '신논현역 6번출구에서 도보3분거리에위치하고 있는 미미옥은겉에서 봤을 땐 브런치카페 느낌이드는 모던한 외관인데알고보면 한식 메뉴를 파는 곳으로반전매력이 있는 식당이었다', '입구쪽에는 테라스석도 있어서4~5월 날씨 좋은 날 딱 요즘같을 때바깥에서 식사를 즐기기에도 좋아보인다저녁시간에도 분위기가 좋을 것 같다테라스에서 즐기는 샤브샤브라니완전 분위기 맛집~', '내부는 차분한 조명과 우드 인테리어가아늑한 분위기로 친구들과의 모임이나데이트를 즐기기에도 좋아보였다', '입구쪽에는 신선하게 준비된샤브용 채소들이 보였는데그 자체로 예쁜 분위기를 만들어내고 있었다신선한 재료들을 즉석에서 준비하여내어주시니 더욱 믿음이 가는 미미옥', '서울 3대 샤브샤브 전문점 미미옥은웰컴 키즈존~어린 아이가 있는 가족도방문하기 편하도록36개월 미만의 아기에게순한 맛의 아기 쌀국수까지 제공해주신다니사장님의 배려가 돋보였다샤브샤브와 함께 곁들임 메뉴가함께 묶여있는 세트메뉴는 개별로주문하는 것보다 1천원 할인되기 때문에다양한 메뉴를 즐기고 싶다면세트메뉴가 합리적~강남역 샤브샤브 맛집은점심 메뉴로 쌀국수, 꼬막비빔국수도있어서 간단하게 식사하기 위해방문하기에도 좋다', '또 인상적이었던 것이전통주 종류가 엄청나게 많다는 것~주류 메뉴판이 따로 있었고사진상에 있는 것보다 훨씬 다양해서선택지가 많았다사장님이 술에 진심이신듯ㅎ우리는 둘이서 배터지게 먹으려고능이 샤브샤브, 육회, 미나리 육전 튀김그리고 오미자/매실 하이볼까지아주 푸짐하게 주문했다ㅋ', '한국의 허브라 불리는 방아 잎을띄운 육수를 샤브샤브를 끓일그릇에 부어주시는데그릇도 너무 정갈하고 예쁘다🥰바닥에는 안

In [ ]:
# Chrome driver 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)


# blog 본문 가져오기 
for i in range(0, len(blog_url_lst)) :
    blog_url = blog_url_lst[i]
    driver.get(blog_url)
    time.sleep(1)
    
    
    # 본문 크롤링 예외 처리 
    try : 
        # iframe 접근 
        driver.switch_to.frame('mainFrame')
        crawling_blog = {}
        
        # 제목 가져오기
        overlays_title = ".se-module.se-module-text.se-title-text"
        html_title = driver.find_element(By.CSS_SELECTOR, overlays_title)
        title = html_title.text
        
        # 작성 날짜 가져오기
        overlays_publishDate = ".se_publishDate.pcol2"
        html_publishDate = driver.find_element(By.CSS_SELECTOR, overlays_publishDate)
        date = html_publishDate.text
        
        # 본문 가져오기
        overlays_content = ".se-component.se-text.se-l-default"
        html_content = driver.find_elements(By.CSS_SELECTOR, overlays_content)
        contents_lst = []
        for content in html_content : 
            contents_lst.append(content.text.replace('\n',''))
        content_str = ''.join(contents_lst)
        
        
        # 크롤링한 내용 담기 
        crawling_blog['title'] = title
        crawling_blog['date'] = date
        crawling_blog['content'] = content_str
        dict[i] = crawling_blog
        
        
        # driver 종료 
        driver.close
        time.sleep(1)
        
    # 에러 발생시 다음 블로그로 이동
    except : 
        print("error",i,title)
        driver.close()
        time.sleep(1)
        
        
        

        
